In [1]:
import re
import sys
import traceback
from concurrent.futures import ThreadPoolExecutor, wait
from datetime import datetime
from queue import Queue

import pymongo
import tweepy
from kipp.decorator import debug_wrapper
from pymongo import MongoClient
from tweepy import API, OAuthHandler

executor = ThreadPoolExecutor(max_workers=10)


# sys.path.append(r'/Users/laisky/repo/laisky/ramjet/ramjet/settings')
sys.path.append(r"/opt/configs/ramjet")
sys.path
import prd

mongo = MongoClient(
    f"mongodb://{prd.MONGO_USER}:{prd.MONGO_PASSWD}@{prd.MONGO_HOST}:{prd.MONGO_PORT}/{prd.MONGO_DB}",
)
tweets = mongo["twitter"]["tweets"]
users = mongo["twitter"]["users"]

auth = OAuthHandler(prd.CONSUMER_KEY, prd.CONSUMER_SECRET)
auth.set_access_token(prd.ACCESS_TOKEN, prd.ACCESS_TOKEN_SECRET)
api = API(auth, wait_on_rate_limit=True, parser=tweepy.parsers.JSONParser())
api.me()


{'id': 1354782101308985349,
 'id_str': '1354782101308985349',
 'name': 'Yksial',
 'screen_name': 'aftergreatest',
 'location': 'Azerbaijan',
 'profile_location': None,
 'description': '🌞🌍🌝🪐🌌',
 'url': 'https://t.co/SqS07XdHHz',
 'entities': {'url': {'urls': [{'url': 'https://t.co/SqS07XdHHz',
     'expanded_url': 'https://talkingsallowed.gov.cn',
     'display_url': 'talkingsallowed.gov.cn',
     'indices': [0, 23]}]},
  'description': {'urls': []}},
 'protected': False,
 'followers_count': 217,
 'friends_count': 165,
 'listed_count': 1,
 'created_at': 'Thu Jan 28 13:23:32 +0000 2021',
 'favourites_count': 334,
 'utc_offset': None,
 'time_zone': None,
 'geo_enabled': False,
 'verified': False,
 'statuses_count': 680,
 'lang': None,
 'status': {'created_at': 'Fri Apr 02 07:32:53 +0000 2021',
  'id': 1377886741806247936,
  'id_str': '1377886741806247936',
  'text': '@txyyss 心情极佳的时候做梦会梦到红辣椒',
  'truncated': False,
  'entities': {'hashtags': [],
   'symbols': [],
   'user_mentions': [{'scr

In [2]:
api.search(q='to:sophyzhang2003', since_id='1368898903500800007', tweet_mode="extended")

{'statuses': [{'created_at': 'Tue Mar 09 09:28:09 +0000 2021',
   'id': 1369218438900506627,
   'id_str': '1369218438900506627',
   'full_text': '@sophyzhang2003 @threadreaderapp',
   'truncated': False,
   'display_text_range': [16, 32],
   'entities': {'hashtags': [],
    'symbols': [],
    'user_mentions': [{'screen_name': 'sophyzhang2003',
      'name': 'Sophy Zhang',
      'id': 868737886270705664,
      'id_str': '868737886270705664',
      'indices': [0, 15]},
     {'screen_name': 'threadreaderapp',
      'name': 'Thread Reader App',
      'id': 895814938995957760,
      'id_str': '895814938995957760',
      'indices': [16, 32]}],
    'urls': []},
   'metadata': {'iso_language_code': 'und', 'result_type': 'recent'},
   'source': '<a href="http://twitter.com/download/android" rel="nofollow">Twitter for Android</a>',
   'in_reply_to_status_id': 1369204682443329538,
   'in_reply_to_status_id_str': '1369204682443329538',
   'in_reply_to_user_id': 868737886270705664,
   'in_reply_to_

In [2]:
api.user_timeline?

In [3]:
api.user_timeline(tweet_mode='extended', max_id=1372203983008276482)

[{'created_at': 'Wed Mar 17 15:11:38 +0000 2021',
  'id': 1372203983008276481,
  'id_str': '1372203983008276481',
  'full_text': '一个更复杂也更完整的链条。\n财政部向银行发行债券来为政府募资，然后财政部成立中投，中投收购汇金，汇金作为银行大股东获取银行股息上交给财政部，财政部用股息偿还债券利息。这就可以解释为什么中国银行一直维持高额派息，以及它们为什么需要不断地上市募资。 https://t.co/QGiKbLQ0LM',
  'truncated': False,
  'display_text_range': [0, 122],
  'entities': {'hashtags': [],
   'symbols': [],
   'user_mentions': [],
   'urls': [],
   'media': [{'id': 1372203954403090440,
     'id_str': '1372203954403090440',
     'indices': [123, 146],
     'media_url': 'http://pbs.twimg.com/media/EwsMOe2VIAgG5lR.jpg',
     'media_url_https': 'https://pbs.twimg.com/media/EwsMOe2VIAgG5lR.jpg',
     'url': 'https://t.co/QGiKbLQ0LM',
     'display_url': 'pic.twitter.com/QGiKbLQ0LM',
     'expanded_url': 'https://twitter.com/hs_yksial/status/1372203983008276481/photo/1',
     'type': 'photo',
     'sizes': {'thumb': {'w': 150, 'h': 150, 'resize': 'crop'},
      'small': {'w': 680, 'h': 538, 'resize': 'fit'},
      'l

In [ ]:
# index

# tweets.create_index([('created_at', pymongo.DESCENDING)])
# tweets.create_index([('id_str', pymongo.DESCENDING)])
# tweets.create_index([('created_at', pymongo.DESCENDING)])
tweets.create_index([('user.id_str', pymongo.DESCENDING)])
# users.create_index([('id', pymongo.DESCENDING)])

In [ ]:
# delete tweets

# @debug_wrapper
def is_status_tobe_delete(tweet):
    if len(tweet.get("entities", {}).get("hashtags", [])) >= 1:
        return False

    if tweet["created_at"] > datetime(2021, 1, 1):
        return False

    if tweet.get("in_reply_to_status_id") is not None:
        return False

    if tweets.count_documents({"in_reply_to_status_id": tweet["id"]}) > 0:
        return False

    return True


@debug_wrapper
def run_delete():
    delete_q = Queue(maxsize=50)
    fs = []
    fs.append(executor.submit(gen_tweet, delete_q))
    for _ in range(10):
        fs.append(executor.submit(delete_tweet, delete_q))

    wait(fs)


@debug_wrapper
def gen_tweet(q: Queue):
    for tweet in tweets.find({"deleted": {"$ne": True}}).sort("created_at", 1):
        q.put(tweet)


@debug_wrapper
def delete_tweet(q: Queue):
    while 1:
        tweet = q.get()
        if not is_status_tobe_delete(tweet):
            print(f"pass tweet {tweet['id']} {tweet['created_at']}")
            continue

        try:
            api.destroy_status(tweet["id"])
        except tweepy.error.TweepError as err:
            if "No status found with that ID." in f"{err}":
                # deleted
                tweets.update_one(
                    {"_id": tweet["_id"]},
                    {"$set": {"deleted": True}},
                )
                print(f"mark {tweet['id']} {tweet['created_at']} deleted")
                continue

            traceback.print_exc()
        except Exception:
            traceback.print_exc()

        tweets.update_one(
            {"_id": tweet["_id"]},
            {"$set": {"deleted": True}},
        )

        #         n_del += 1
        print("delete", tweet["id"], tweet["created_at"])


#         if n % 1000 == 0:
#             print(f">> scan {n} statuses")
#         if n_del % 100 == 0:
#             print(f">> delete {n_del} statuses")


run_delete()


In [ ]:
# download videos

import requests


requests.get('https://video.twimg.com/ext_tw_video/1180858517940396035/pu/vid/864x486/84gdcqfeV-ud1LgC.mp4?tag=10')

In [ ]:
# download image
from pathlib import Path

import requests

dirpath = r"/var/www/uploads/twitter"
# dirpath = r'/Users/laisky/Downloads'


def download_images_for_tweet(tweet):
    for img in tweet["entities"]["media"]:
        with requests.get(img["media_url_https"] + ":orig") as r:
            if r.status_code != 200:
                print(f"download error: [{r.status_code}]{r.content}")
                continue

            fpath = Path(dirpath, img["media_url_https"].split("/")[-1])
            if fpath.is_file():
                continue

            with open(fpath, "wb") as f:
                f.write(r.content)

            print("tweet img ok", tweet["id"], fpath)


def download_images():
    for tweet in tweets.find(
        {"entities.media": {"$exists": 1}}, no_cursor_timeout=True
    ).sort("_id", -1):
        download_images_for_tweet(tweet)


download_images()


In [ ]:
# download related tweets


def get_tweet_text(tweet: Dict[str, any]) -> str:
    return tweet.get("full_text") or tweet.get("text")


def twitter_api_parser(tweet: Dict[str, any]) -> Dict[str, any]:
    """Parse tweet document got from twitter api"""
    reg_topic = re.compile(r"[\b|\s]#(\S+)")
    tweet["topics"] = reg_topic.findall(get_tweet_text(tweet).replace(".", "_"))
    tweet["created_at"] = datetime.datetime.strptime(
        tweet["created_at"], "%a %b %d %H:%M:%S +0000 %Y"
    )

    # replace url
    t = get_tweet_text(tweet)
    if tweet.get("entities"):
        # parse entities media
        entities = tweet.get("extended_entities") or tweet.get("entities")
        if "media" in entities:
            for media in tweet["entities"]["media"]:
                for surl in ["url", "display_url"]:
                    durl = media.get("media_url_https") or media["media_url"]
                    t = t.replace(surl, durl)

        # parse entities urls
        if "urls" in tweet["entities"]:
            for d in tweet["entities"]["urls"]:
                for surl in ["url", "display_url"]:
                    eurl = d["expanded_url"]
                    t = t.replace(surl, eurl)

        tweet["text"] = t

    return tweet


def gen_related_tweets(
    tweetCol: pymongo.collection.Collection, tweet: Dict[str, any]
) -> Generator[str, None, None]:
    related_ids = []
    tweet.get("in_reply_to_status_id") and related_ids.append(
        tweet["in_reply_to_status_id"]
    )
    tweet.get("retweeted_status") and related_ids.append(
        tweet["retweeted_status"]["id"]
    )
    tweet.get("quoted_status") and related_ids.append(tweet["quoted_status"]["id"])
    for _id in filter(lambda id_: not tweetCol.find_one({"id": id_}), related_ids):
        yield _id


def save_relate_tweets(status):
    related_ids = []
    status.get("in_reply_to_status_id") and related_ids.append(
        status["in_reply_to_status_id"]
    )
    status.get("retweeted_status") and related_ids.append(
        status["retweeted_status"]["id"]
    )
    status.get("quoted_status") and related_ids.append(status["quoted_status"]["id"])
    related_ids = filter(lambda id_: not tweets.find_one({"id": id_}), related_ids)

    for id_ in related_ids:
        try:
            docu = api.get_status(id_, tweet_mode="extended")
        except Exception as err:
            print(f"load tweet {id_} got error: {err}")
        else:
            print(f"save tweet [{docu['user']['screen_name']}]{docu['id']}")
            save_tweet(docu)
            save_relate_tweets(docu)


def save_tweet(docu):
    docu = twitter_api_parser(docu)
    tweets.update_one({"id": docu["id"]}, {"$set": docu}, upsert=True)


def download_relate_tweets():
    for tweet in tweets.find(no_cursor_timeout=True).sort("_id", -1):
        save_relate_tweets(tweet)


# download_relate_tweets()


In [ ]:
# replace all urls


def replace_to_laisky_url(url):
    url = url.replace(
        "http://pbs.twimg.com/media/", "https://s3.laisky.com/uploads/twitter/"
    )
    url = url.replace(
        "https://pbs.twimg.com/media/", "https://s3.laisky.com/uploads/twitter/"
    )

    return url


def replace_media_urls(tweet):
    for ee in ["entities", "extended_entities"]:
        for media in tweet.get(ee, {}).get("media", []):
            surl = media.get("url", "")
            durl = media.get("media_url_https", "")
            durl = replace_to_laisky_url(durl)
            if durl != "":
                tweet["text"] = tweet["text"].replace(surl, durl)


def replace_short_urls(tweet):
    for media in tweet.get("entities", {}).get("urls", []):
        surl = media.get("url", "")
        durl = media.get("expanded_url", "")
        if durl != "":
            tweet["text"] = tweet["text"].replace(surl, durl)


def replace_all_urls():
    i = 0
    for tweet in tweets.find(no_cursor_timeout=True).sort("_id", 1):
        replace_media_urls(tweet)
        replace_short_urls(tweet)

        #         i += 1
        #         print(tweet['text'])
        #         if i > 10:
        #             return
        #         else:
        #             continue

        tweets.update_one({"id": tweet["id"]}, {"$set": tweet})


replace_all_urls()


In [ ]:
# save users

def save_users():
    for tweet in tweets.find(no_cursor_timeout=True).sort("_id", 1):
        if tweet.get("user") is None:
            continue

        user = tweet["user"]
        users.update_one({"id": user["id"]}, {"$set": user}, upsert=True)


save_users()

In [ ]:
# remove duplicate tweets

def remove_duplicate_tweets():
    for tweet in tweets.find(no_cursor_timeout=True).sort("_id", -1):
        tweets.delete_many({"id_str": tweet["id_str"], "_id": {"$ne": tweet["_id"]}})


def add_id_str():
    for tweet in tweets.find(no_cursor_timeout=True).sort("_id", -1):
        tweets.update_one(
            {"_id": tweet["_id"]},
            {"$set": {"id_str": str(tweet["id"]), "id": int(tweet["id"])}},
        )


def add_user_id_str():
    for docu in users.find(no_cursor_timeout=True).sort("_id", -1):
        users.update_one(
            {"_id": docu["_id"]},
            {"$set": {"id_str": str(docu["id"]), "id": int(docu["id"])}},
        )


add_id_str()
add_user_id_str()
remove_duplicate_tweets()

In [4]:
s = 'https://twitter.com/aftergreatest/status/1382892663754551298'

